In [1]:
import re
import os
import requests
import zipfile
import math
import pandas as pd
import numpy as np
from tqdm import tqdm

In [13]:
root_url = 'http://www.kdnuggets.com/data_mining_course/data/'
root_data = 'data'
train_data_file = 'ALL_AML_gr.thr.train.csv'
data_file = 'ALL_AML_train_processed.zip'

In [9]:
def unzip_file(file_addr,output_dir,remove=False) :
    with zipfile.ZipFile(file_addr,"r") as zip_ref:
        zip_ref.extractall(output_dir)
    if remove :  
        os.remove(file_addr)
    return os.listdir(output_dir)

def download_file(url,output_file,unzip=False,output_dir='') :
    if not os.path.exists(data_file) :
        # Streaming, so we can iterate over the response.
        r = requests.get(url, stream=True)

        # Total size in bytes.
        total_size = int(r.headers.get('content-length', 0)); 
        block_size = 1024
        wrote = 0 
        with open(output_file, 'wb') as f:
            for data in tqdm(r.iter_content(block_size), total=math.ceil(total_size//block_size) , unit='KB', unit_scale=True):
                wrote = wrote  + len(data)
                f.write(data)
        if total_size != 0 and wrote != total_size:
            print("ERROR, something went wrong")
    if unzip :
        return unzip_file(output_file,output_dir,True)
    return output_file

files = download_file(root_url + data_file,data_file,True,root_data)

381KB [00:18, 20.7KB/s]                                                                                                


In [17]:
pd.read_csv(os.path.join(root_data,files[0])) \
            .to_csv(os.path.join(root_data,train_data_file),index=False)

In [23]:
data = pd.read_csv(os.path.join(root_data,train_data_file))

In [28]:
data['fold variation'] = data.apply(lambda row: max(row[1:]) - min(row[1:]),axis=1)

In [32]:
max(data['fold variation'])

15980

In [40]:
# data.sort_values('fold variation', axis=0,ascending=False)
data['fold variation'][data['fold variation'].idxmax()]

15980

In [48]:
rows_with_large_fold = data.loc[data['fold variation'] == max(data['fold variation'])]
print('Largest fold diffrence is {} and {} rows have it'.format(max(rows_with_large_fold['fold variation']),len(rows_with_large_fold)))

Maximum fold variation is 15980 and 17 rows have it


In [49]:
rows_with_low_fold = data.loc[data['fold variation'] == min(data['fold variation'])]
print('Lowest fold diffrence is {} and {} rows have it'.format(min(rows_with_low_fold['fold variation']),len(rows_with_low_fold)))

Lowest fold diffrence is 0 and 476 rows have it
